In [ ]:
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout 
from keras.layers.normalization import BatchNormalization 
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from google.colab import auth, drive
from keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from numpy import asarray
import zipfile
import numpy as np
from keras.callbacks import TensorBoard # new! 

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_txt_files(file_type):
  path = None
  if file_type is 1:
    path = 'drive/MyDrive/TAA/one-indexed-files-notrash_train.txt' 
  else:
    path = 'drive/MyDrive/TAA/one-indexed-files-notrash_val.txt'
  file = open(path, mode='r')
  classification = []
  archives = []
  labels = []
  for line in file:
    lines = line.replace('\n', '').split(' ')
    archives.append(lines[0])
    labels.append(int(lines[1]) - 1) 
#    classification.append({'classification': int(lines[1]), 'archive': lines[0]})
  return (archives, labels)

In [ ]:
  def get_image_as_array(image_path, gray_scale = False):
    if gray_scale:
      image = img_to_array(load_img(image_path, color_mode="grayscale"))
    else:
      image = img_to_array(load_img(image_path)) 
    return img_to_array(image)
    #return asarray(image)

In [ ]:
def resize_image(archive, image_path, new_width, new_height, destination_folder):
  img = Image.open(image_path) # image extension *.png,*.jpg
  img = img.resize((new_height, new_width), Image.ANTIALIAS)
  img.save(destination_folder + archive) # format may what u want ,*.png,*jpg,*.gif

In [ ]:
(train_archives, train_labels) = get_txt_files(1)
(validation_archives, validation_labels) = get_txt_files(0)

In [ ]:
# for archive in train_archives:
#   path = 'drive/MyDrive/TAA/imgs/' + archive
#   resize_image(archive, path, 230, 307, 'drive/MyDrive/TAA/imgs_230_x_307/')

# for archive in validation_archives:
#   path = 'drive/MyDrive/TAA/imgs/' + archive
#   resize_image(archive, path, 230, 307, 'drive/MyDrive/TAA/imgs_230_x_307/')

In [ ]:
#  !unrar X 'drive/MyDrive/TAA/imgs.rar' 'drive/MyDrive/TAA'

In [ ]:
train_values = []
validation_values = []

for archive in train_archives:
  path = 'drive/MyDrive/TAA/imgs_230_x_307/' + archive
  train_values.append(get_image_as_array(path))

for archive in validation_archives:
  path = 'drive/MyDrive/TAA/imgs_230_x_307/' + archive
  validation_values.append(get_image_as_array(path))

In [ ]:
train_values = np.array(train_values)
validation_values = np.array(validation_values)

In [ ]:
train_values.shape

(1768, 230, 307, 3)

In [ ]:
validation_values.shape

(328, 230, 307, 3)

In [ ]:
# train_values = train_values.reshape(1768, 196608).astype('float32')
# validation_values = validation_values.reshape(328, 196608).astype('float32')

In [ ]:
train_values /= 255
validation_values /= 255

In [ ]:
n_classes = 6
train_labels = keras.utils.to_categorical(train_labels, n_classes)
validation_labels = keras.utils.to_categorical(validation_labels, n_classes)

In [ ]:
train_labels

array([[0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()

model.add(Conv2D(64, 3, activation='relu', input_shape=(230, 307, 3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Conv2D(128, 3, activation='relu'))
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(512, (3,3), activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Conv2D(1024, (3,3), activation='relu'))
model.add(Conv2D(1024, 3, activation='relu'))
model.add(Conv2D(1024, 3, activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 228, 305, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 226, 303, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 113, 151, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 113, 151, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 111, 149, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 109, 147, 128)     147584    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 107, 145, 128)     1

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(
          train_values, train_labels,
          batch_size=64,
          epochs=100,
          verbose=1,
          validation_data=(validation_values, validation_labels)
          )

Epoch 1/100
28/28 [==============================] - 4043s 144s/step - loss: 5.4990 - accuracy: 0.2423 - val_loss: 151.7828 - val_accuracy: 0.1402
Epoch 2/100
28/28 [==============================] - 4022s 144s/step - loss: 2.6372 - accuracy: 0.2452 - val_loss: 24.9887 - val_accuracy: 0.1402
Epoch 3/100
28/28 [==============================] - 3998s 143s/step - loss: 2.0831 - accuracy: 0.2648 - val_loss: 5.8581 - val_accuracy: 0.2530
Epoch 4/100
28/28 [==============================] - 4016s 144s/step - loss: 1.6887 - accuracy: 0.3376 - val_loss: 3.0520 - val_accuracy: 0.2348
Epoch 5/100
28/28 [==============================] - 4009s 143s/step - loss: 1.6438 - accuracy: 0.3867 - val_loss: 3.1494 - val_accuracy: 0.2256
Epoch 6/100
 4/28 [===>..........................] - ETA: 55:03 - loss: 1.5911 - accuracy: 0.4040